In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


In [34]:
df=pd.read_csv("melb_data.csv")

In [35]:
df.head(1)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0


In [36]:
y = df.Price
X = df.drop(['Price'], axis=1)
## tahminden ayırmak

In [37]:
## datayi ayırmak
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [38]:
## unique verileri az olanları seçelim
cat_cols=[cname for cname in X_train_full.columns if X_train_full[cname].nunique() <10 and X_train_full[cname].dtype=="object"]

In [39]:
num_cols=[cname for cname in X_train_full.columns if X_train_full[cname].dtype in  ['int64', 'float64']]

In [40]:
##sadece seiçili sutünları seç

In [41]:
my_cols=cat_cols+num_cols
X_train= X_train_full[my_cols].copy()
X_valid=X_valid_full[my_cols].copy()

In [42]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


In [43]:
## Pipeline 3 adımda oluşturma

In [44]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [45]:
num_transfor= SimpleImputer(strategy='constant')
## verileri ön işlemesi
cat_transfer=Pipeline(steps=[    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
## Bunde önişlemesi
processor=ColumnTransformer(
    transformers=[
        ('num', num_transfor, num_cols),
        ('cat', cat_transfer, cat_cols)
    ])

In [46]:
## random forest model

In [47]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=100, random_state=0)
## bir dizi karar ağacının bir araya getirilmesiyle oluşan bir öğrenme algoritmasıdır ve regresyon problemleri için kullanılabilir.

In [48]:
## OLUŞTUR  VE DEGERLENDİR PİPELİNE

In [49]:
from sklearn.metrics import mean_absolute_error

In [50]:
my_pipeline=Pipeline(steps=[('preprocessor', processor),
                              ('model', model)])
## training data fit model
my_pipeline.fit(X_train,y_train)

## doğrulama verilerinin ön işlemesi tahmin alma

preds=my_pipeline.predict(X_valid)

## modeli degerlendir

skor=mean_absolute_error(y_valid,preds)
print("Skor",skor)

Skor 160679.18917034855


In [51]:
## Pipeline kodunu temizler ve hata önler

In [52]:
##CROSS VALİDASYON


In [53]:
## her denemenin puanlarının birbirine yakın görünüp görünmediğini görebilirsiniz. Her deney aynı sonuçları veriyorsa tek bir doğrulama seti muhtemelen yeterlidir.

In [54]:
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = df[cols_to_use]

In [59]:
y=df.Price

In [63]:
from sklearn.model_selection import cross_val_score


In [64]:
##Puanlama parametresi raporlanacak model kalitesi ölçüsünü seçer: bu durumda negatif ortalama mutlak hatayı (MAE) ->tutarlılığı sağlar
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [65]:
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

MAE scores:
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


In [66]:
print("Ortalama MAE score:")
print(scores.mean())

Ortalama MAE score:
277707.3795913405


In [67]:
##Çapraz doğrulamanın kullanılması, model kalitesine ilişkin çok daha iyi bir ölçüm sağlar ve ek olarak kodumuzu temizleme avantajı sağlar:
#

In [68]:
## GRADİENT BOOSTİNG

In [69]:
#modelleri bir topluluğa yinelemeli olarak eklemek için döngülerden geçen bir yöntemdir

In [71]:
from sklearn.model_selection import train_test_split


In [73]:
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = df[cols_to_use]

In [74]:
y=df.Price

In [82]:
x_train,x_valid,y_train,y_valid=train_test_split(X,y)

In [83]:
from xgboost import XGBRegressor

In [84]:
my_model = XGBRegressor()
my_model.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [85]:
## regresyon problemleri için XGBoost'un özel bir uygulamasıdır ve bağımlı değişkenin sürekli (sayısal) olduğu durumlar için kullanılır.

In [86]:
from sklearn.metrics import mean_absolute_error

In [87]:
tahmins=my_model.predict(x_valid)
print("Ortalama Err"+str(mean_absolute_error(tahmins, y_valid)))

Ortalama Err242638.0729197349


In [88]:
my_model2 = XGBRegressor(n_estimators=500)
my_model2.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [90]:
my_model3 = XGBRegressor(n_estimators=500)
my_model3.fit(x_train, y_train,
             early_stopping_rounds=5,
             eval_set=[(x_valid, y_valid)],
             verbose=False)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [91]:
##Tüm verilerinizi bir modele sığdırmak istiyorsanız, n_estimators'ı erken durdurmayla çalıştırdığınızda en uygun bulduğunuz değere ayarlayın.

In [92]:
my_model4 = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model4.fit(x_train, y_train,
             early_stopping_rounds=5,
             eval_set=[(x_valid, y_valid)],
             verbose=False)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [93]:
##Çalışma süresinin önemli olduğu daha büyük veri kümelerinde modellerinizi daha hızlı oluşturmak için njob kullan

In [95]:
my_model5 = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model5.fit(x_train, y_train,
             early_stopping_rounds=5,
             eval_set=[(x_valid, y_valid)],
             verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

In [96]:
##XGBoost, standart tablo verileriyle (görüntüler ve videolar gibi daha egzotik veri türlerinin aksine, Pandas DataFrames'te depoladığınız veri türü) çalışmak için
## önde gelen bir yazılım kitaplığıdır. Dikkatli parametre ayarıyla yüksek doğruluklu modelleri eğitebilirsiniz.

In [97]:
## DATA LEAKAGE


##Eğitim ve doğrulama verilerinin dikkatli bir şekilde ayrılması, tren-test kontaminasyonunu önleyebilir ve işlem hatları bu ayrımın uygulanmasına yardımcı olabilir.
## Benzer şekilde, dikkat, sağduyu ve veri araştırmasının bir kombinasyonu, hedef sızıntının belirlenmesine yardımcı olabilir


In [98]:
##verilerin kullanılabilir hale geldiği zamanlama veya kronolojik sıraya göre düşünmek önemlidir.


In [113]:
card=pd.read_csv("AER_credit_card_data.csv", true_values = ['yes'], false_values = ['no'])

In [114]:
y=card.card

In [115]:
X = card.drop(['card'], axis=1) ## select tahmins

In [116]:
print("sutün sayisi:", X.shape[0])
X.head()

sutün sayisi: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


In [117]:
## şüpheli degiskenleri ayıralım

In [118]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

In [119]:
print('Kart almayan ve harcama yapmayanların oranı: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Kart alıp harcama yapmayanların oranı: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Kart almayan ve harcama yapmayanların oranı: 1.00
Kart alıp harcama yapmayanların oranı: 0.02


In [120]:
## Sızdıran tahmin edicileri veri kümesinden çıkarın

In [121]:
potansiyal_sizan=['expenditure', 'share', 'active', 'majorcards']

In [123]:
X2 = X.drop(potansiyal_sizan, axis=1)


In [125]:
## degerlendir
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))


In [126]:



cv_scores=cross_val_score(my_pipeline,X2,y, cv=5,
                            scoring='accuracy')

In [127]:
print("Cross dogrulama %f" % cv_scores.mean())

Cross dogrulama 0.834710
